# **Occupancy Grid Mapping**

Occupancy grid mapping is a probabilistic method used in robotics to build a map of the environment while simultaneously determining the robot's location within that map. It represents the environment as a grid of cells, each containing the probability that the cell is occupied by an obstacle.

In this Lab, we'll implement an occupancy grid mapping algorithm, simulate a robot exploring an unknown environment, and handle sensor noise using Bayesian update rules.

## **Setting Up the Environment**
First, let's import the necessary libraries.

In [1]:
import pygame
import numpy as np
import math
import random
import sys
from scipy.special import expit

pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
random.seed(42)
np.random.seed(42)

---

## **Defining the Occupancy Grid**
We'll define the occupancy grid as a 2D NumPy array, where each cell holds the log-odds of occupancy.

In [2]:
GRID_WIDTH, GRID_HEIGHT = 50, 50 
CELL_SIZE = 10  

# Initialize the occupancy grid with zero log-odds (unknown)
occupancy_grid = np.zeros((GRID_WIDTH, GRID_HEIGHT))

SCREEN_WIDTH = GRID_WIDTH * CELL_SIZE
SCREEN_HEIGHT = GRID_HEIGHT * CELL_SIZE

---

## **Log-Odds Representation**
Using log-odds helps in numerical stability and simplifies the Bayesian update equations.
 - **Log-odds of occupancy:**
 
![office layout](figures/eq1.png)

 - **Probability from log-odds:**
 
![office layout](figures/eq2.png)


### **Update Equation**
For each cell, the log-odds is updated as:

![office layout](figures/eq3.png)


- 𝑙𝑡: Updated log-odds at time 
- 𝑙𝑡-1 : Previous log-odds.
- 𝑃(cell occupied∣measurement): Sensor model probability.
- 𝑙0: Prior log-odds (usually 0 for unknown).

###  **Example**
![office layout](figures/grid_mapping.png)

---

## **Simulating the Robot and Environment**
### Environment Map
We'll create a simple environment with obstacles.

In [3]:
def create_environment(grid_width, grid_height):
    environment = np.zeros((grid_width, grid_height))
    
    # Add obstacles
    environment[10:15, 10:40] = 1
    environment[20:35, 30:35] = 1
    environment[40:45, 5:25] = 1
    return environment

true_map = create_environment(GRID_WIDTH, GRID_HEIGHT)

## Robot Simulation
We'll simulate a robot moving randomly in the environment.

In [4]:
class Robot:
    def __init__(self, environment):
        self.env = environment
        self.x = random.randint(0, GRID_WIDTH - 1)
        self.y = random.randint(0, GRID_HEIGHT - 1)
        while self.env[self.x, self.y] == 1:
            self.x = random.randint(0, GRID_WIDTH - 1)
            self.y = random.randint(0, GRID_HEIGHT - 1)
        self.orientation = random.uniform(0, 2 * math.pi)
        self.path = [(self.x, self.y)]
    
    def move(self):
        # Simple random walk
        dx = random.choice([-1, 0, 1])
        dy = random.choice([-1, 0, 1])
        nx, ny = self.x + dx, self.y + dy
        
        # Check bounds and obstacles
        if 0 <= nx < GRID_WIDTH and 0 <= ny < GRID_HEIGHT:
            if self.env[nx, ny] == 0:
                self.x, self.y = nx, ny
                self.path.append((self.x, self.y))
                if dx != 0 or dy != 0:
                    self.orientation = math.atan2(dy, dx)
                    
    def sense(self):
        # Simulate a sensor reading (ideal for now)
        max_range = 10
        measurements = []
        angles = np.linspace(-math.pi / 4, math.pi / 4, 5) + self.orientation
        for angle in angles:
            for r in range(1, max_range + 1):
                nx = int(self.x + r * math.cos(angle))
                ny = int(self.y + r * math.sin(angle))
                if 0 <= nx < GRID_WIDTH and 0 <= ny < GRID_HEIGHT:
                    measurements.append((nx, ny))
                    if self.env[nx, ny] == 1:
                        break  # Obstacle detected
                else:
                    break  # Out of bounds
        return measurements

---

## **Handling Sensor Noise**
### **Introducing Noise into Sensor Readings**
We'll modify the sense function to include sensor noise. (You can use this in case you want your robot to have some noise in its sensopr readings)

In [5]:
class NoisyRobot(Robot):
    def sense(self):
        # Simulate a sensor reading with noise
        max_range = 10
        measurements = []
        angles = np.linspace(-math.pi / 4, math.pi / 4, 5) + self.orientation
        for angle in angles:
            for r in range(1, max_range + 1):
                nx = int(self.x + r * math.cos(angle))
                ny = int(self.y + r * math.sin(angle))
                if 0 <= nx < GRID_WIDTH and 0 <= ny < GRID_HEIGHT:
                    # Introduce false positives and negatives
                    if random.random() < 0.1:
                        continue  # Missed detection (false negative)
                    measurements.append((nx, ny))
                    if self.env[nx, ny] == 1:
                        if random.random() > 0.1:
                            break  # Correct detection
                        else:
                            continue  # False negative
                else:
                    break  # Out of bounds
        return measurements

---

## **Implementing Sensor Models**
We'll define a simple sensor model for occupancy probabilities.

### Inverse Sensor Model
The inverse sensor model provides 𝑃(cell occupied∣measurement) for each cell.

In [6]:
def inverse_sensor_model(cell, robot_pos, env):
    x_cell, y_cell = cell
    x_robot, y_robot = robot_pos
    if env[x_cell, y_cell] == 1:
        return 0.9  # High probability of being occupied
    else:
        return 0.3  # Low probability of being occupied

## **Implementing Filters to Handle Uncertainty**
We'll adjust the inverse sensor model to account for sensor noise.

In [7]:
def inverse_sensor_model_noisy(cell, robot_pos, env):
    x_cell, y_cell = cell
    x_robot, y_robot = robot_pos
    if env[x_cell, y_cell] == 1:
        return 0.7  # Slightly lower due to noise
    else:
        return 0.4  # Slightly higher due to noise

In [8]:
def draw(screen, occupancy_grid, robot):
    screen.fill((255, 255, 255)) 
    
    # Draw occupancy grid
    for x in range(GRID_WIDTH):
        for y in range(GRID_HEIGHT):
            l = occupancy_grid[x, y]
            p = 1 - 1 / (1 + math.exp(l))  # Convert log-odds to probability
            color = (255 * (1 - p), 255 * (1 - p), 255 * (1 - p))
            rect = pygame.Rect(y * CELL_SIZE, x * CELL_SIZE, CELL_SIZE, CELL_SIZE)
            pygame.draw.rect(screen, color, rect)
    
    pygame.draw.circle(screen, (0, 0, 255), (int(robot.y * CELL_SIZE + CELL_SIZE / 2), int(robot.x * CELL_SIZE + CELL_SIZE / 2)), CELL_SIZE // 2)
    
    for pos in robot.path:
        pygame.draw.circle(screen, (0, 255, 0), (int(pos[1] * CELL_SIZE + CELL_SIZE / 2), int(pos[0] * CELL_SIZE + CELL_SIZE / 2)), 2)


In [9]:
pygame.init()

screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Occupancy Grid Mapping")
clock = pygame.time.Clock()

robot = Robot(true_map)
running = True
num_steps = 10000  # Set higher for continuous simulation

step = 0
while running and step < num_steps:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            
    robot.move()
    measurements = robot.sense()
    
    # Update occupancy grid
    for cell in measurements:
        # Compute the inverse sensor model
        prob = inverse_sensor_model(cell, (robot.x, robot.y), true_map)
        # Update log-odds
        lx, ly = cell
        l_prior = occupancy_grid[lx, ly]
        l_sensor = math.log(prob / (1 - prob))
        occupancy_grid[lx, ly] = l_prior + l_sensor - 0  # l0 = 0
    
    draw(screen, occupancy_grid, robot)
    pygame.display.flip()
    clock.tick(60)  
    step += 1

pygame.quit()




---

# **Simultaneous Localization and Mapping (SLAM)**
Simultaneous Localization and Mapping (SLAM) refers to creating a map of an unknown environment while simultaneously determining location. SLAM is commonly associated with developing autonomous vehicles and robotics. Other application areas of SLAM also include augmented / virtual reality (AR / VR), for tracking objects and the user, agriculture, for tracking fields and enabling data-driven decision-making, and medicine, especially in minimally invasive surgeries where slender instruments are snaked through small incisions.
## **Typles of SLAM**
### **Filter Based SLAM**
Filter based SLAM treats the SLAM problem as a state-estimation problem, where the state encompasses information about the current position and map. The state is recursively updated by a filter, which estimates the current position and map based on actions and measurements. As more data is collected, the estimate is augmented and refined.

Some filter based SLAM algorithms are:
#### **Kalman Filter**
This is a recursive algorithm that works in two steps: a prediction step and an update step. The prediction step makes a prediction for the state based on the dynamics of the system, and the update steps reconcile the prediction with the measurements to produce an improved estimate of the state. The information from the prediction and measurements are weighed based on their uncertainty / (co)-variance. The Kalman Filter algorithm assumes a linear world with Gaussian error / noise; in fact, the Kalman Filter is the optimal / best estimator in a linear, Gaussian world when optimizing for mean square error between the estimated and true state. However, a limitation of the Kalman Filter is that linearity and / or Gaussian error / noise do not always apply.

#### **1. Extended Kalman Filter SLAM(EKS)**
The Extended Kalman Filter (EKS) extends the Kalman Filter and allows us to relax the linearity assumption by using linearization. The technique involves using a first-order Taylor expansion around the estimate and keeps most of the Kalman Filter mechanics. The EKS algorithm is a very popular technique for nonlinear state estimation due to its flexibility and efficiency. However, EKS can also introduce large errors in the estimate and lead to suboptimal performance.



---

In [5]:
pygame.init()

screen_width = 800
screen_height = 600
screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption("EKF SLAM Simulation")

WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
DARK_GRAY = (50, 50, 50)
LIGHT_GRAY = (200, 200, 200)

# Robot parameters
robot_pos = np.array([400, 300])
robot_orientation = 0  
robot_velocity = 5  # Pixels per frame
turn_rate = 0.1  
robot_radius = 10 

# Initial Landmark Positions
landmarks = [
    np.array([100, 100]),
    np.array([700, 500]),
    np.array([400, 500]),
    np.array([600, 200]),
]

obstacles = [
    pygame.Rect(200, 150, 100, 50),
    pygame.Rect(500, 300, 150, 100),
    pygame.Rect(300, 400, 50, 150),
]

# EKF SLAM Parameters
mu = np.array([400, 300, 0])  # Initial belief about state [x, y, theta]
Sigma = np.eye(3) * 0.1  # Initial covariance

# Control noise covariance
R = np.diag([0.1, 0.1, 0.05])

# Measurement noise covariance
Q = np.diag([0.5, 0.5])

# Visited areas (initially all dark)
visited = np.full((screen_width, screen_height), False)

# Radius for lighting up areas
light_radius = 50


def motion_model(mu, u):
    """
    Applies the motion model given control inputs.
    u: Control input [velocity, steering_angle]
    """
    x, y, theta = mu
    v, w = u
    x += v * math.cos(theta)
    y += v * math.sin(theta)
    theta += w

    return np.array([x, y, theta])


def ekf_prediction(mu, Sigma, u):
    """Predict step for EKF"""
    # Jacobian of the motion model
    theta = mu[2]
    G = np.array([
        [1, 0, -u[0] * math.sin(theta)],
        [0, 1, u[0] * math.cos(theta)],
        [0, 0, 1]
    ])
    
    mu_bar = motion_model(mu, u)
    Sigma_bar = G @ Sigma @ G.T + R
    return mu_bar, Sigma_bar


def ekf_update(mu, Sigma, z, landmark_pos):
    """Update step for EKF"""
    # Measurement prediction
    delta = landmark_pos - mu[:2]
    q = np.dot(delta, delta)
    z_hat = np.array([
        math.sqrt(q),
        math.atan2(delta[1], delta[0]) - mu[2]
    ])
    
    # Jacobian of the measurement model
    H = np.array([
        [-delta[0] / math.sqrt(q), -delta[1] / math.sqrt(q), 0],
        [delta[1] / q, -delta[0] / q, -1]
    ])
    
    # Kalman Gain
    S = H @ Sigma @ H.T + Q
    K = Sigma @ H.T @ np.linalg.inv(S)
    
    # Measurement residual
    y = z - z_hat
    y[1] = (y[1] + np.pi) % (2 * np.pi) - np.pi  # Normalize angle
    
    # Update state
    mu = mu + K @ y
    Sigma = (np.eye(len(Sigma)) - K @ H) @ Sigma
    
    return mu, Sigma


def draw_robot(x, y, orientation):
    pygame.draw.circle(screen, GREEN, (int(x), int(y)), robot_radius)


def add_new_landmark(mu, landmarks, obstacles):
    """
    Adds a new landmark if a new feature is detected.
    """
    new_landmark = mu[:2] + np.random.randn(2) * 20  # Add some noise for new landmark position
    min_distance = 50  # Minimum distance to consider it a new feature

    # Check if the new landmark is sufficiently far from existing landmarks
    for landmark in landmarks:
        if np.linalg.norm(new_landmark - landmark) < min_distance:
            return landmarks  # Do not add if too close to an existing landmark

    # Check if the new landmark collides with any obstacles
    for obstacle in obstacles:
        if obstacle.collidepoint(new_landmark):
            return landmarks  # Do not add if it collides with an obstacle

    # Add new landmark
    landmarks.append(new_landmark)
    return landmarks


def check_collision(x, y, obstacles):
    """
    Check if the robot's position collides with any obstacles.
    """
    robot_rect = pygame.Rect(x - robot_radius, y - robot_radius, robot_radius * 2, robot_radius * 2)
    for obstacle in obstacles:
        if robot_rect.colliderect(obstacle):
            return True
    return False


def avoid_collision(mu, u, obstacles):
    """
    Modify control to avoid collisions if necessary.
    """
    potential_mu = motion_model(mu, u)
    future_rect = pygame.Rect(potential_mu[0] - robot_radius, potential_mu[1] - robot_radius, robot_radius * 2, robot_radius * 2)
    # Check if the robot is within a safe distance to avoid collision
    for obstacle in obstacles:
        expanded_obstacle = obstacle.inflate(30, 30)  # Expand obstacle by a margin to ensure early avoidance
        if future_rect.colliderect(expanded_obstacle) or not (15 <= potential_mu[0] <= screen_width - 15 and 15 <= potential_mu[1] <= screen_height - 15):
            # If collision is detected or almost detected (including borders), adjust the direction
            u[0] = 0  # Stop forward movement
            u[1] = turn_rate  # Turn to avoid obstacle or boundary
            return u
    return u


def main():
    global robot_pos, robot_orientation, mu, Sigma, landmarks, visited
    clock = pygame.time.Clock()
    running = True

    while running:
        screen.fill(BLACK)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
                
        u = [robot_velocity, 0]


        # Modify control to avoid collisions and boundaries
        u = avoid_collision(mu, u, obstacles)

        # EKF Prediction
        mu_bar, Sigma_bar = ekf_prediction(mu, Sigma, u)

        # Update state after collision check
        mu, Sigma = mu_bar, Sigma_bar

        # Update visited areas
        x, y = int(mu[0]), int(mu[1])
        for i in range(max(0, x - light_radius), min(screen_width, x + light_radius)):
            for j in range(max(0, y - light_radius), min(screen_height, y + light_radius)):
                if (i - x) ** 2 + (j - y) ** 2 <= light_radius ** 2:
                    visited[i, j] = True

        # Draw visited areas
        visited_surface = pygame.Surface((screen_width, screen_height), pygame.SRCALPHA)
        visited_surface.fill((0, 0, 0, 0))
        for i in range(screen_width):
            for j in range(screen_height):
                if visited[i, j]:
                    visited_surface.set_at((i, j), WHITE)
        screen.blit(visited_surface, (0, 0))

        # Simulate measurements to landmarks
        for landmark in landmarks:
            delta = landmark - mu[:2]
            distance = np.linalg.norm(delta)
            bearing = math.atan2(delta[1], delta[0]) - mu[2]
            z = np.array([distance + np.random.randn() * Q[0, 0], bearing + np.random.randn() * Q[1, 1]])
            mu, Sigma = ekf_update(mu, Sigma, z, landmark)

        # Add new landmarks as the robot explores
        landmarks = add_new_landmark(mu, landmarks, obstacles)

        # Update robot position (for visualization)
        robot_pos = mu[:2]
        robot_orientation = mu[2]

        for obstacle in obstacles:
            pygame.draw.rect(screen, BLUE, obstacle)

        for landmark in landmarks:
            pygame.draw.circle(screen, RED, landmark.astype(int), 5)

        # Draw robot
        draw_robot(robot_pos[0], robot_pos[1], robot_orientation)

        pygame.display.flip()
        clock.tick(60)
    pygame.quit()

if __name__ == "__main__":
    main()


---

#### 3. **Unscented Kalman Filter (UKS)**
The Unscented Kalman Filter (UKS) extends the Kalman Filter and allows us to relax the linearity assumption by approximating a probability distribution. The technique involves using an unscented transformation that calculates statistics, such as the mean or variance, for a random variable that undergoes a nonlinear transformation (by sampling sigma points that represent the distribution of the state). The UKS can provide more accurate estimates for the state, especially in more nonlinear systems, but can be more computationally expensive.

#### 4. **Particle Filter**
The Particle Filter represents the estimate as a set of particles, where each particle represents a possible state; particles with higher weights are deemed to be more reflective of the true state. As more information is collected, the particles are re-weighted based on their match to the observed measurements. A key strength of the Particle Filter is that it has very few assumptions of the underlying system, allowing for complexity, and the ability to handle non-linear systems and non-Gaussian error / noise (unlike the Kalman Filter). As such, it’s very flexible and can be used in a wide variety of situations. However, the Particle Filter does also have limitations such as dimensionality (i.e., a lot of particles may be needed to accurately express the state distribution), divergence (i.e., the particles can diverge from the true state), and complexity (i.e., the computation required can be intensive). There are variants that build upon the Particle Filter and address these limitations.

In general, filter based SLAM shines in structured environments where the dynamics are known — in this setting, it can be computationally efficient and potentially provide real-time estimates.

---

### **Graph Based SLAM**
Graph based SLAM treats the SLAM problem as a graph problem, where position information is represented by the nodes and the map is derived with the edges. In the robotics space, it’s common to use pose-graph optimization. In a pose graph, the nodes represent poses and landmarks and the edges represent constraints between them. New nodes are added as new poses and landmarks are detected, and constraints connect sequential nodes with information about the movement. For example, moving from one point to another would be represented with two nodes, each containing information about the pose and landmarks measured, connected by an edge, containing information about the motion and other observations. In constructing the pose graph, we can also add edges between nodes if they are similar enough, indicating a return to some previous poses and landmarks. When two nodes are very similar, it provides information to update the pose graph and potentially add new edges indicating their closeness (this is also sometimes called detecting a loop closure).

The objective is to optimize the graph based on measurements observed by minimizing the errors in the poses, landmarks, and constraints. Algorithms for graph based SLAM aim to perform the optimization efficiently; some examples are:
#### **1. Square Root Smoothing And Mapping (Square Root SAM)**
This algorithm solves the graph optimization with factorization. The algorithm leverages the property that the SLAM graph is sparse, meaning that each node is only connected to a few other nodes / has only a few edges to and from it. The reason why SLAM graphs are sparse is because node connections / edges are only formed between sequential poses or when there’s a return to a previous pose. The latter reason, a return to a previous pose, is relatively infrequent compared to the number of different poses we track. Furthermore, there’s typically a high threshold for deeming two poses to be the same. When we represent the graph as a matrix, where the rows and columns represent the nodes and the entries represent the edge connection between nodes, the matrix will have many zero elements, thus being a sparse matrix. Optimization on a sparse matrix can be made efficient using factorization techniques from linear algebra, which is what the Square Root SAM algorithm does, to provide more numerically stable and efficient estimates.

#### **2. Incremental Smoothing and Mapping (iSAM)**
This algorithm leverages factorization similar to Square Root SAM and allows for incremental updates for the estimate as new measurements are collected. Unlike Square Root SAM which is a batch-based approach, meaning that it waits for all available data before optimizing, iSAM does it on the fly, allowing it to be more real-time. Furthermore, since it’s updating at each increment as opposed to in batches, it reduces the computational complexity and memory needed.

Note: There are also Deep learning-based SLAM that has been developed.

In [22]:
import pygame
import random
import math
import networkx as nx
import numpy as np
from scipy.optimize import least_squares
import logging

# -----------------------------
#         Configuration
# -----------------------------

# Initialize Logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Initialize Pygame
pygame.init()

# Screen dimensions
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Optimized Incremental Smoothing and Mapping (iSAM) Simulation")

# Colors
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
DARK_GRAY = (50, 50, 50)
LIGHT_GRAY = (200, 200, 200)
ROBOT_COLOR = (0, 255, 0)
OBSTACLE_COLOR = (255, 0, 0)
NODE_COLOR = (0, 0, 255)
EDGE_COLOR = (255, 255, 0)
LOOP_CLOSURE_COLOR = (255, 0, 255)  # Magenta for loop closures
OPTIMIZED_NODE_COLOR = (0, 255, 255)  # Cyan for optimized nodes

# Robot parameters
robot_pos = np.array([WIDTH // 2, HEIGHT // 2], dtype=float)
robot_angle = random.uniform(0, 360)  # Degrees
robot_speed = 2.0 
robot_radius = 10
sensing_radius = 60  

# Buffer distance for boundaries
boundary_buffer = robot_radius + 5  # 5 pixels additional buffer

# Map parameters
map_surface = pygame.Surface((WIDTH, HEIGHT))
map_surface.fill(BLACK)

# Obstacle parameters
num_obstacles = 15  # Increased number of obstacles for complexity
obstacles = []

# Pose Graph Parameters
pose_graph = nx.Graph()
node_id = 0
previous_node = node_id
pose_graph.add_node(node_id, pos=robot_pos.copy(), angle=robot_angle)
node_id += 1

# Dictionary to store previous positions and angles of nodes for correction
previous_positions = {}
previous_angles = {}

# Initialize loop closure tracking
last_loop_closure_pos = robot_pos.copy()
last_loop_closure_angle = robot_angle

# Toggle for displaying pose graph
show_pose_graph = True

# Trajectory for visualization
trajectory = []

# Optimization in progress flag
optimization_in_progress = False

# -----------------------------
#         Functions
# -----------------------------

def generate_obstacles():
    """Generate random rectangular obstacles ensuring no overlap with the robot's starting area."""
    for _ in range(num_obstacles):
        attempts = 0
        while True:
            attempts += 1
            if attempts > 100:
                logging.warning("Exceeded maximum attempts to place obstacles without overlap.")
                break
            w = random.randint(30, 100)
            h = random.randint(30, 100)
            x = random.randint(0, WIDTH - w)
            y = random.randint(0, HEIGHT - h)
            obstacle_rect = pygame.Rect(x, y, w, h)
            # Ensure obstacles do not overlap the robot's starting position
            robot_rect = pygame.Rect(robot_pos[0] - robot_radius*4, robot_pos[1] - robot_radius*4,
                                     robot_radius*8, robot_radius*8)
            overlap = False
            for obstacle in obstacles:
                if obstacle.colliderect(obstacle_rect):
                    overlap = True
                    break
            if not obstacle_rect.colliderect(robot_rect) and not overlap:
                obstacles.append(obstacle_rect)
                break

def draw_obstacles(surface):
    """Draw all obstacles on the given surface."""
    for obstacle in obstacles:
        pygame.draw.rect(surface, OBSTACLE_COLOR, obstacle)

def check_collision(pos):
    """Check if the robot's new position collides with obstacles or boundaries."""
    robot_rect = pygame.Rect(pos[0] - robot_radius, pos[1] - robot_radius,
                             robot_radius*2, robot_radius*2)
    # Check screen boundaries with buffer
    if not (boundary_buffer <= pos[0] <= WIDTH - boundary_buffer and
            boundary_buffer <= pos[1] <= HEIGHT - boundary_buffer):
        return True
    # Check collision with obstacles
    for obstacle in obstacles:
        if robot_rect.colliderect(obstacle):
            return True
    return False

def is_position_free(pos):
    """Check if the position is free from obstacles and within boundaries."""
    robot_rect = pygame.Rect(pos[0] - robot_radius, pos[1] - robot_radius,
                             robot_radius*2, robot_radius*2)
    # Check screen boundaries with buffer
    if not (boundary_buffer <= pos[0] <= WIDTH - boundary_buffer and
            boundary_buffer <= pos[1] <= HEIGHT - boundary_buffer):
        return False
    # Check collision with obstacles
    for obstacle in obstacles:
        if robot_rect.colliderect(obstacle):
            return False
    return True

def add_node_with_position_tracking(graph, node_id, pos, angle):
    """Add a node to the pose graph with position and angle tracking."""
    graph.add_node(node_id, pos=pos.copy(), angle=angle)
    previous_positions[node_id] = pos.copy()
    previous_angles[node_id] = angle

def detect_loop_closure(current_pos, current_angle, threshold=50, min_distance_since_last=150, angle_threshold=30):
    """
    Detect loop closure by checking proximity and orientation alignment with previous nodes.
    Returns the node ID if a loop closure is detected, else None.
    """
    global last_loop_closure_pos, last_loop_closure_angle
    distance_since_last = np.linalg.norm(current_pos - last_loop_closure_pos)
    angle_diff_since_last = abs(current_angle - last_loop_closure_angle)
    angle_diff_since_last = min(angle_diff_since_last, 360 - angle_diff_since_last)  # Account for wrap-around
    if distance_since_last < min_distance_since_last and angle_diff_since_last < angle_threshold:
        return None  # Too soon for another loop closure

    for n in pose_graph.nodes:
        if n == previous_node:
            continue
        pos = pose_graph.nodes[n]['pos']
        angle = pose_graph.nodes[n].get('angle', 0)
        distance = np.linalg.norm(current_pos - pos)
        angle_diff = abs(current_angle - angle)
        angle_diff = min(angle_diff, 360 - angle_diff)
        if distance < threshold and angle_diff < angle_threshold:
            logging.info(f"Loop closure detected between node {node_id-1} and node {n}.")
            last_loop_closure_pos = current_pos.copy()
            last_loop_closure_angle = current_angle
            return n
    return None

def store_previous_positions_angles(graph):
    """Store previous positions and angles before optimization."""
    for n in graph.nodes:
        previous_positions[n] = graph.nodes[n]['pos'].copy()
        previous_angles[n] = graph.nodes[n]['angle']

def optimize_graph(graph, recent_nodes=10):
    """
    Perform pose graph optimization on the last 'recent_nodes' nodes.
    Minimizes the error based on movement constraints and obstacle proximity.
    """
    global optimization_in_progress
    if optimization_in_progress:
        return  # Prevent overlapping optimizations
    optimization_in_progress = True

    nodes = list(graph.nodes)
    subset_nodes = nodes[-recent_nodes:] if len(nodes) > recent_nodes else nodes
    subset_graph = graph.subgraph(subset_nodes)
    subset_nodes = list(subset_graph.nodes)
    subset_poses = np.array([graph.nodes[n]['pos'] for n in subset_nodes])
    subset_angles = np.array([graph.nodes[n]['angle'] for n in subset_nodes])

    if len(subset_nodes) < 2:
        # Not enough nodes to optimize
        optimization_in_progress = False
        return

    def error_function(x):
        errors = []
        # Extract positions and angles
        updated_poses = x[:len(subset_nodes)*2].reshape((-1, 2))
        updated_angles = x[len(subset_nodes)*2:].reshape((-1, 1)).flatten()

        # Position and Orientation constraints based on edges (odometry and loop closures)
        for edge in subset_graph.edges(data=True):  # Corrected to include data=True
            n1, n2, attrs = edge
            if n1 in subset_nodes and n2 in subset_nodes:
                idx1 = subset_nodes.index(n1)
                idx2 = subset_nodes.index(n2)
                pos1 = updated_poses[idx1]
                pos2 = updated_poses[idx2]
                angle1 = updated_angles[idx1]
                angle2 = updated_angles[idx2]
                # Odometry constraint: distance should be approximately robot_speed * number of frames between nodes
                desired_distance = robot_speed * 1  # Assuming 1 frame between nodes
                actual_distance = np.linalg.norm(pos2 - pos1)
                errors.append(actual_distance - desired_distance)

                # Orientation consistency constraint: angle difference should be small
                desired_angle_diff = (angle2 - angle1) % 360
                desired_angle_diff = min(desired_angle_diff, 360 - desired_angle_diff)
                errors.append(desired_angle_diff)

        # Proximity penalties to obstacles
        for pos in updated_poses:
            min_distance = float('inf')
            for obstacle in obstacles:
                obstacle_center = np.array([obstacle.centerx, obstacle.centery])
                distance = np.linalg.norm(pos - obstacle_center)
                # Minimum allowed distance from obstacle center
                obstacle_min_distance = math.sqrt((obstacle.width / 2) ** 2 + (obstacle.height / 2) ** 2) + robot_radius + 5
                if distance < obstacle_min_distance:
                    min_distance = min(min_distance, obstacle_min_distance - distance)
            # Add a penalty if too close to any obstacle
            if min_distance < float('inf'):
                errors.append(min_distance * 10)  # Scale penalty as needed

        return errors

    # Store previous positions and angles before optimization
    store_previous_positions_angles(graph)

    # Flatten subset poses and angles for optimization
    x0 = np.hstack((subset_poses.flatten(), subset_angles))

    try:
        # Perform optimization using 'trf' method
        res = least_squares(error_function, x0, verbose=0, method='trf')
    except Exception as e:
        logging.error(f"Optimization failed: {e}")
        optimization_in_progress = False
        return

    # Update poses and angles with optimized values
    optimized_poses = res.x[:len(subset_nodes)*2].reshape((-1, 2))
    optimized_angles = res.x[len(subset_nodes)*2:].reshape((-1, 1)).flatten()

    for idx, n in enumerate(subset_nodes):
        # Store previous position and angle before updating
        previous_positions[n] = graph.nodes[n]['pos'].copy()
        previous_angles[n] = graph.nodes[n]['angle']
        # Update with optimized values
        graph.nodes[n]['pos'] = optimized_poses[idx]
        graph.nodes[n]['angle'] = optimized_angles[idx] % 360  # Ensure angle stays within [0, 360)
        # Optionally, mark optimized nodes for visualization
        graph.nodes[n]['optimized'] = True

    logging.info("Pose graph optimization completed.")
    optimization_in_progress = False

def verify_and_correct_nodes(graph):
    """Verify node positions after optimization and revert if inside obstacles."""
    for n in graph.nodes:
        pos = graph.nodes[n]['pos']
        if not is_position_free(pos):
            logging.warning(f"Node {n} moved into an obstacle. Reverting to previous position and angle.")
            # Revert to the previous position and angle
            graph.nodes[n]['pos'] = previous_positions[n].copy()
            graph.nodes[n]['angle'] = previous_angles[n]
            # Remove the 'optimized' flag if present
            if 'optimized' in graph.nodes[n]:
                del graph.nodes[n]['optimized']

def snap_position_outside_obstacle(pos):
    """Optional: Snap the robot's position outside the nearest obstacle."""
    for obstacle in obstacles:
        if obstacle.colliderect(pygame.Rect(pos[0] - robot_radius, pos[1] - robot_radius,
                                           robot_radius*2, robot_radius*2)):
            # Closest point on obstacle's perimeter
            closest_x = np.clip(pos[0], obstacle.left, obstacle.right)
            closest_y = np.clip(pos[1], obstacle.top, obstacle.bottom)
            closest_point = np.array([closest_x, closest_y])
            direction = pos - closest_point
            if np.linalg.norm(direction) == 0:
                # Choose a random direction if exactly on the center
                direction = np.array([random.uniform(-1, 1), random.uniform(-1, 1)])
            direction = direction / np.linalg.norm(direction)
            # Snap to just outside the obstacle
            new_pos = closest_point + direction * (robot_radius + 5)
            return new_pos
    return pos

def draw_graph(surface, graph, max_nodes=50, alpha=150):
    """Draw the pose graph edges and nodes on the given surface."""
    if not show_pose_graph:
        return

    # Create a transparent surface for edges
    edge_surface = pygame.Surface((WIDTH, HEIGHT), pygame.SRCALPHA)

    # Get the last 'max_nodes' nodes
    nodes = list(graph.nodes)
    if len(nodes) > max_nodes:
        nodes_subset = nodes[-max_nodes:]
    else:
        nodes_subset = nodes
    subset_graph = graph.subgraph(nodes_subset)

    # Draw edges on the transparent surface
    for edge in subset_graph.edges(data=True):  # Corrected to include data=True
        n1, n2, attrs = edge
        pos1 = graph.nodes[n1]['pos']
        pos2 = graph.nodes[n2]['pos']
        if attrs.get('loop', False):
            color = LOOP_CLOSURE_COLOR + (alpha,)  # Add alpha channel
            width = 3
        else:
            color = EDGE_COLOR + (alpha,)  # Add alpha channel
            width = 2
        pygame.draw.line(edge_surface, color, pos1, pos2, width)

    # Blit the transparent edge surface onto the main surface
    surface.blit(edge_surface, (0, 0))

def draw_nodes(surface, graph, max_nodes=50):
    """Draw the pose graph nodes on the given surface."""
    if not show_pose_graph:
        return

    # Create a transparent surface for nodes
    node_surface = pygame.Surface((WIDTH, HEIGHT), pygame.SRCALPHA)

    # Get the last 'max_nodes' nodes
    nodes = list(graph.nodes)
    if len(nodes) > max_nodes:
        nodes_subset = nodes[-max_nodes:]
    else:
        nodes_subset = nodes
    subset_graph = graph.subgraph(nodes_subset)

    # Draw nodes
    for n in subset_graph.nodes:
        pos = graph.nodes[n]['pos']
        if graph.nodes[n].get('optimized', False):
            color = OPTIMIZED_NODE_COLOR  # Different color for optimized nodes
        else:
            color = NODE_COLOR
        pygame.draw.circle(node_surface, color, pos.astype(int), 3)

    # Blit the transparent node surface onto the main surface
    surface.blit(node_surface, (0, 0))

def draw_robot_orientation(surface, pos, angle, length=15):
    """Draw the robot's orientation as a line extending from its center."""
    rad_angle = math.radians(angle)
    end_x = pos[0] + length * math.cos(rad_angle)
    end_y = pos[1] - length * math.sin(rad_angle)
    pygame.draw.line(surface, WHITE, pos, (end_x, end_y), 2)

def sense():
    """Visualize the robot's sensing radius by filling the explored area with white."""
    pygame.draw.circle(map_surface, WHITE, robot_pos.astype(int), sensing_radius, 0)  # Filled circle

def add_edge_if_valid(graph, from_node, to_node, loop=False):
    """Add an edge between two nodes if it doesn't already exist."""
    if graph.has_edge(from_node, to_node):
        return
    graph.add_edge(from_node, to_node, loop=loop)

def handle_events():
    """Handle Pygame events such as quitting and key presses."""
    global show_pose_graph
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            exit()
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_g:
                show_pose_graph = not show_pose_graph
                logging.info(f"Pose graph display toggled to {show_pose_graph}.")

# -----------------------------
#         Initialization
# -----------------------------

generate_obstacles()
logging.info(f"Generated {num_obstacles} obstacles.")

# Initialize pose graph with position and angle tracking
add_node_with_position_tracking(pose_graph, node_id, robot_pos.copy(), robot_angle)
# Initialize last node position and angle
last_node_pos = robot_pos.copy()
last_node_angle = robot_angle
node_id += 1

# Initialize robot state
STATE_MOVING_FORWARD = 'moving_forward'
STATE_TURNING_LEFT = 'turning_left'
STATE_TURNING_RIGHT = 'turning_right'
STATE_RECOVERING = 'recovering'
robot_state = STATE_MOVING_FORWARD

# Initialize turning attempt counter
turning_attempts = 0
max_turning_attempts = 18  # For 5-15 degrees per frame turning (total ~90 degrees)

# Main loop control
running = True
clock = pygame.time.Clock()

# -----------------------------
#         Main Loop
# -----------------------------

while running:
    clock.tick(60)  # 60 FPS
    handle_events()

    # Autonomous navigation logic using state machine
    if robot_state == STATE_MOVING_FORWARD:
        # Calculate multiple sensor positions for better collision detection
        sensor_angles = [0, 30, -30, 60, -60]  # Front, front-left, front-right, side-left, side-right
        obstacle_ahead = False
        for a in sensor_angles:
            rad = math.radians(robot_angle + a)
            sensor_x = robot_pos[0] + (robot_radius + 10) * math.cos(rad)
            sensor_y = robot_pos[1] - (robot_radius + 10) * math.sin(rad)
            sensor_pos = np.array([sensor_x, sensor_y])
            if check_collision(sensor_pos):
                obstacle_ahead = True
                logging.debug(f"Obstacle detected at sensor angle {robot_angle + a:.2f} degrees.")
                break

        if obstacle_ahead:
            # Decide turning direction
            turn_direction = random.choice(['left', 'right'])
            if turn_direction == 'left':
                robot_state = STATE_TURNING_LEFT
                logging.info("State changed to TURNING_LEFT.")
            else:
                robot_state = STATE_TURNING_RIGHT
                logging.info("State changed to TURNING_RIGHT.")
        else:
            # Move forward
            rad_angle = math.radians(robot_angle)
            new_x = robot_pos[0] + robot_speed * math.cos(rad_angle)
            new_y = robot_pos[1] - robot_speed * math.sin(rad_angle)
            new_pos = np.array([new_x, new_y])

            if not check_collision(new_pos):
                robot_pos = new_pos
                trajectory.append(robot_pos.copy())
                if len(trajectory) > 500:  # Reduced trajectory length for performance
                    trajectory.pop(0)

                # Check if the robot has moved enough to add a new node
                distance_moved = np.linalg.norm(robot_pos - last_node_pos)
                if distance_moved >= 20:  # min_distance
                    if is_position_free(robot_pos):
                        # Add new node with position and angle tracking
                        add_node_with_position_tracking(pose_graph, node_id, robot_pos.copy(), robot_angle)
                        # Add edge from previous node to current node
                        add_edge_if_valid(pose_graph, previous_node, node_id)
                        # Detect loop closure
                        loop_node = detect_loop_closure(robot_pos, robot_angle)
                        if loop_node is not None:
                            add_edge_if_valid(pose_graph, node_id, loop_node, loop=True)
                        # Update previous node information
                        previous_node = node_id
                        node_id += 1
                        last_node_pos = robot_pos.copy()
                        last_node_angle = robot_angle
                        logging.info(f"Added node {node_id - 1} at position {robot_pos} with angle {robot_angle:.2f} degrees.")
                    else:
                        logging.warning("Attempted to add a node inside an obstacle. Skipping node addition.")
            else:
                # Collision occurred unexpectedly, initiate turning
                turn_direction = random.choice(['left', 'right'])
                if turn_direction == 'left':
                    robot_state = STATE_TURNING_LEFT
                    logging.info("Collision detected. State changed to TURNING_LEFT.")
                else:
                    robot_state = STATE_TURNING_RIGHT
                    logging.info("Collision detected. State changed to TURNING_RIGHT.")

    elif robot_state == STATE_TURNING_LEFT:
        # Turn left with a randomized angle for variability
        turn_angle = random.uniform(5, 15)  # Degrees per frame
        robot_angle = (robot_angle + turn_angle) % 360
        turning_attempts += 1
        logging.debug(f"Turning left by {turn_angle:.2f} degrees. Total attempts: {turning_attempts}")

        if turning_attempts >= max_turning_attempts:
            robot_state = STATE_RECOVERING
            turning_attempts = 0
            logging.info("Completed TURNING_LEFT. State changed to RECOVERING.")

    elif robot_state == STATE_TURNING_RIGHT:
        # Turn right with a randomized angle for variability
        turn_angle = random.uniform(5, 15)  # Degrees per frame
        robot_angle = (robot_angle - turn_angle) % 360
        turning_attempts += 1
        logging.debug(f"Turning right by {turn_angle:.2f} degrees. Total attempts: {turning_attempts}")

        if turning_attempts >= max_turning_attempts:
            robot_state = STATE_RECOVERING
            turning_attempts = 0
            logging.info("Completed TURNING_RIGHT. State changed to RECOVERING.")

    elif robot_state == STATE_RECOVERING:
        # Attempt to move forward again
        robot_state = STATE_MOVING_FORWARD
        logging.info("State changed to MOVING_FORWARD from RECOVERING.")

    # Sense surroundings (fill explored area with white)
    sense()

    # Perform pose graph optimization at set intervals
    frame_count = pygame.time.get_ticks() // (1000 // 60)  # Approximate frame count
    optimization_interval = 240  # Optimize every 240 frames (~4 seconds at 60 FPS)
    if frame_count % optimization_interval == 0:
        optimize_graph(pose_graph, recent_nodes=10)  # Optimize the last 10 nodes for performance
        verify_and_correct_nodes(pose_graph)         # Verify node positions

    # Draw the map
    screen.blit(map_surface, (0, 0))

    # Draw the graph edges and nodes
    draw_graph(screen, pose_graph, max_nodes=50, alpha=150)  # Reduced max_nodes for performance
    draw_nodes(screen, pose_graph, max_nodes=50)            # Reduced max_nodes for performance

    # Draw obstacles on top of edges and nodes
    draw_obstacles(screen)

    # Draw the robot
    pygame.draw.circle(screen, ROBOT_COLOR, robot_pos.astype(int), robot_radius)
    draw_robot_orientation(screen, robot_pos.astype(int), robot_angle)

    # Draw the trajectory (optional)
    if len(trajectory) > 1:
        pygame.draw.lines(screen, WHITE, False, [pos.astype(int) for pos in trajectory], 1)

    # Update the display
    pygame.display.flip()

pygame.quit()


2024-10-19 21:01:25,397 - INFO - Generated 15 obstacles.
2024-10-19 21:01:25,567 - INFO - Added node 2 at position [419.71164955 296.61608628] with angle 9.74 degrees.
2024-10-19 21:01:25,734 - INFO - Added node 3 at position [439.4232991  293.23217256] with angle 9.74 degrees.
2024-10-19 21:01:25,901 - INFO - Added node 4 at position [459.13494865 289.84825884] with angle 9.74 degrees.
2024-10-19 21:01:26,070 - INFO - Added node 5 at position [478.8465982  286.46434512] with angle 9.74 degrees.
2024-10-19 21:01:26,239 - INFO - Added node 6 at position [498.55824774 283.0804314 ] with angle 9.74 degrees.
2024-10-19 21:01:26,356 - INFO - State changed to TURNING_RIGHT.
2024-10-19 21:01:26,658 - INFO - Completed TURNING_RIGHT. State changed to RECOVERING.
2024-10-19 21:01:26,674 - INFO - State changed to MOVING_FORWARD from RECOVERING.
2024-10-19 21:01:26,941 - INFO - Loop closure detected between node 6 and node 7.
2024-10-19 21:01:26,942 - INFO - Added node 7 at position [478.56950418 

error: display Surface quit

: 

---